<a href="https://colab.research.google.com/github/rasoco/master_data_science/blob/master/TFM/Data%20Preparation/DataPreparation_BiciMadProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BiciMad Project  - Master Data Science KSchool Ed.019**



## Conect with GPU of Google 



In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU no encontrada')
print('Encontrada GPU: {}'.format(device_name))

Encontrada GPU: /device:GPU:0


In [2]:
# !pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
drive

## Data Preparation

### Madrid Stations Dataset

#### Load of the libraries


In [4]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import folium # To display maps with coordinates and markers.
import geopy # For geocoding and reverse geocoding.


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


#### Load of the Madrid Stations dataset


In [5]:
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/originalbikestations.csv", sep=';', encoding='utf-8') #Firt all, active google Drive. Decimal is okey
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Número            216 non-null    object 
 1   Gis_X             216 non-null    object 
 2   Gis_Y             216 non-null    object 
 3   Fecha de Alta     216 non-null    object 
 4   Distrito          216 non-null    object 
 5   Barrio            216 non-null    object 
 6   Calle             216 non-null    object 
 7   Nº Finca          216 non-null    object 
 8   Tipo de Reserva   216 non-null    object 
 9   Número de Plazas  216 non-null    int64  
 10  Unnamed: 10       0 non-null      float64
 11  LONGITUD          216 non-null    float64
 12  LATITUD           216 non-null    float64
 13  DIRECCION         216 non-null    object 
dtypes: float64(3), int64(1), object(10)
memory usage: 23.8+ KB


In [6]:
del df['Unnamed: 10']

In [7]:
df['Year'] = pd.DatetimeIndex(df['Fecha de Alta']).year
year = df['Year']
df = df.rename(columns = {'Número':'id','Número de Plazas':'Anclajes'}) #change de name of column Número
df.head()

,id,Gis_X,Gis_Y,Fecha de Alta,Distrito,Barrio,Calle,Nº Finca,Tipo de Reserva,Anclajes,LONGITUD,LATITUD,DIRECCION,Year
0,001 a,"440443,61","4474290,65",04/12/2019,01 CENTRO,01-06 SOL,"ALCALA, CALLE, DE",2,BiciMAD,30,-3.701998,40.417111,"CALLE DE ALCALA, 2",2019
1,001 b,"440480,56","4474301,74",06/02/2020,01 CENTRO,01-06 SOL,"ALCALA, CALLE, DE",6,BiciMAD,30,-3.701564,40.417213,"CALLE DE ALCALA, 6",2020
2,2,"440134,83","4474678,23",23/06/2014,01 CENTRO,01-05 UNIVERSIDAD,"MIGUEL MOYA, CALLE, DE",1,BiciMAD,24,-3.705674,40.420580,"CALLE DE MIGUEL MOYA, 1",2014
3,3,"440012,98","4475760,68",23/06/2014,07 CHAMBERÍ,07-02 ARAPILES,"CONDE DEL VALLE DE SUCHIL, PLAZA, DEL",2,BiciMAD,18,-3.707212,40.430323,"PLAZA DEL CONDE DEL VALLE DE SUCHIL, 2",2014
4,4,"440396,4","4475565,36",23/06/2014,01 CENTRO,01-05 UNIVERSIDAD,"MANUELA MALASAÑA, CALLE, DE",3,BiciMAD,24,-3.702674,40.428591,"CALLE DE MANUELA MALASAÑA, 3",2014


In [8]:
df.columns

Index(['id', 'Gis_X', 'Gis_Y', 'Fecha de Alta', 'Distrito', 'Barrio', 'Calle',
       'Nº Finca', 'Tipo de Reserva', 'Anclajes', 'LONGITUD', 'LATITUD',
       'DIRECCION', 'Year'],
      dtype='object')

In [9]:
df['ids'] = df['id'].str.rstrip('ampliacionb') #to get only number values
# df_stations = df_stations[onlycolumns]
df['ids'] = df['ids'].astype(int).sort_values() #change values ids and order
df.head()

,id,Gis_X,Gis_Y,Fecha de Alta,Distrito,Barrio,Calle,Nº Finca,Tipo de Reserva,Anclajes,LONGITUD,LATITUD,DIRECCION,Year,ids
0,001 a,"440443,61","4474290,65",04/12/2019,01 CENTRO,01-06 SOL,"ALCALA, CALLE, DE",2,BiciMAD,30,-3.701998,40.417111,"CALLE DE ALCALA, 2",2019,1
1,001 b,"440480,56","4474301,74",06/02/2020,01 CENTRO,01-06 SOL,"ALCALA, CALLE, DE",6,BiciMAD,30,-3.701564,40.417213,"CALLE DE ALCALA, 6",2020,1
2,2,"440134,83","4474678,23",23/06/2014,01 CENTRO,01-05 UNIVERSIDAD,"MIGUEL MOYA, CALLE, DE",1,BiciMAD,24,-3.705674,40.420580,"CALLE DE MIGUEL MOYA, 1",2014,2
3,3,"440012,98","4475760,68",23/06/2014,07 CHAMBERÍ,07-02 ARAPILES,"CONDE DEL VALLE DE SUCHIL, PLAZA, DEL",2,BiciMAD,18,-3.707212,40.430323,"PLAZA DEL CONDE DEL VALLE DE SUCHIL, 2",2014,3
4,4,"440396,4","4475565,36",23/06/2014,01 CENTRO,01-05 UNIVERSIDAD,"MANUELA MALASAÑA, CALLE, DE",3,BiciMAD,24,-3.702674,40.428591,"CALLE DE MANUELA MALASAÑA, 3",2014,4


In [10]:
df['ids'].unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  22,  23,  24,  25,  26,  27,
        29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
        42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,
        55,  56,  57,  58,  59,  60,  61,  62,  63,  65,  67,  68,  69,
        70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
        83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,
        96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108,
       109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121,
       122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134,
       135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147,
       148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160,
       161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173,
       174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 18

#### Only Stations 2019

In [11]:
stations_2019 = df[df['Year'] <= 2019] #save the 2019 stations data for use to jan18
stations_2019['idunplug_station'] = stations_2019['ids']
stations_2019['idplug_station'] = stations_2019['ids']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


### January 18 BiciMad Dataset

#### Load of Libraries

In [12]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import sklearn 

#### Acquiring data

In [13]:
downloaded = drive.CreateFile({'id':'1dBJtPkHMvyF5Pmeu-w8z-Y91smROB_6b'})
downloaded.GetContentFile('jan18.json')
# downloaded.GetContentFile('jan18_clean.json')

In [14]:
print(os.getcwd)

<built-in function getcwd>


#### Reading data

In [15]:
jan18 = pd.read_json("jan18.json",encoding="utf-8", lines=True, dtype='frame')# open de file

In [16]:
jan18.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263702 entries, 0 to 263701
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   _id               263702 non-null  object
 1   user_day_code     263702 non-null  object
 2   idplug_base       263702 non-null  int64 
 3   user_type         263702 non-null  int64 
 4   idunplug_base     263702 non-null  int64 
 5   travel_time       263702 non-null  int64 
 6   idunplug_station  263702 non-null  int64 
 7   ageRange          263702 non-null  int64 
 8   idplug_station    263702 non-null  int64 
 9   unplug_hourTime   263702 non-null  object
 10  zip_code          263702 non-null  object
dtypes: int64(7), object(4)
memory usage: 22.1+ MB


In [17]:
# Save of position in _id column 
position = dict(enumerate(jan18['_id'].values))
position
type(position)
ids = []
# Save of values in its position 
ids = []
for d, v in position.items():
    ids += list(v.values())
#Save in the column _id
jan18['_id'] = ids 

In [18]:
na_columns = ['_id', 'user_day_code', 'idunplug_station', 'idunplug_base', 'idplug_station', 'idplug_base',
   'user_type','travel_time','unplug_hourTime', 'ageRange', 'zip_code']
jan18 = jan18[na_columns]
jan18 = jan18.rename(columns = {'_id':'id'})

# jan18.columns = ['_id', 'user_day_code', 'idunplug_base','idunplug_station','idplug_base', 'idplug_station',
#        'user_type','travel_time','unplug_hourTime', 'ageRange', 'zip_code']

In [19]:
# jan18.to_csv('jan18_clean.csv') # to save the dataset

In [20]:
jan18.head(1000)

,id,user_day_code,idunplug_station,idunplug_base,idplug_station,idplug_base,user_type,travel_time,unplug_hourTime,ageRange,zip_code
0,5a505acd2f384319304ed614,b00665a845be18ed9f036c2d72def1ef2fa2c49365cb64...,6,14,7,1,1,284,{'$date': '2018-01-01T00:00:00.000+0100'},5,28010
1,5a505acd2f384319304ed615,65a82ac6353d9e8b25c17a49bb46d09426757b40946370...,24,21,117,4,1,666,{'$date': '2018-01-01T00:00:00.000+0100'},0,
2,5a505acd2f384319304ed616,51b7da733980586e1f847b61cc4af9c0268fc2d09df63e...,24,19,117,19,1,662,{'$date': '2018-01-01T00:00:00.000+0100'},0,
3,5a505acd2f384319304ed617,0af09bb0e23045fb7328123044822a0eb9cf154d644a96...,82,2,110,1,1,708,{'$date': '2018-01-01T00:00:00.000+0100'},3,28005
4,5a505acd2f384319304ed619,500e6780660a1fb237f8e5410992296141a46b141ed3ff...,169,3,58,5,1,171,{'$date': '2018-01-01T00:00:00.000+0100'},4,28004
...,...,...,...,...,...,...,...,...,...,...,...
995,5a505ad12f384319304eda71,9a81bb7bb1387b1f8b8b1a7aa6ca1480bce52f9f200017...,17,13,149,1,1,521,{'$date': '2018-01-01T06:00:00.000+0100'},4,28003
996,5a505ad12f384319304eda73,1720f4676d55c2dff9707ad03792e832943de8c20faa27...,166,24,103,23,1,338,{'$date': '2018-01-01T06:00:00.000+0100'},3,28009
997,5a505ad12f384319304eda72,8c088f94ff94c5dbed094c45f7e80210a13910017a0b49...,5,19,170,3,1,1113,{'$date': '2018-01-01T06:00:00.000+0100'},0,
998,5a505ad12f384319304eda74,7bf4c6de14f6289c6fb33663b93fa5f723a7fc5a080e6e...,14,22,26,3,1,341,{'$date': '2018-01-01T06:00:00.000+0100'},4,28001


In [21]:
# jan18[jan18['idplug_station'] == 2008] #show the result

In [22]:
jan18 = jan18.drop(jan18[jan18.idplug_station == 2008].index) #deleting values 2008

In [23]:
jan18.isna().sum()

id                  0
user_day_code       0
idunplug_station    0
idunplug_base       0
idplug_station      0
idplug_base         0
user_type           0
travel_time         0
unplug_hourTime     0
ageRange            0
zip_code            0
dtype: int64

In [24]:
# About null 
jan18.isnull().sum().sum()

0

### Unification processing Dataset (Jan18 and Stations_2019)

#### Unified process

In [25]:
new_dataset = pd.merge(jan18, stations_2019, on='idunplug_station')
new_dataset = new_dataset.rename(columns={'Gis_X':'Gis_Xidunplug',
       'Gis_Y':'Gis_Yidunplug', 'Fecha de Alta':'Fecha de Alta_idunplug', 'Distrito':'Distrito_idunplug', 'Barrio':'Barrio_idunplug', 'Calle':'Calle_idunplug', 'Nº Finca':'Nº Finca_idunplug',
        'Anclajes':'Anclajes_idunplug', 'Longitud':'Longitud_idunplug', 'Latitud':'Latitud_idunplug', 'DIRECCION':'DIRECCION_idunplug',
       'Year':'Year_idunplug', 'Locationlist':'Locationlist_idunplug', 'X':'X_idunplug', 'Y':'Y_idunplug'}) #change de name of the columns

del new_dataset['ids']
another_dataset = pd.merge(jan18,stations_2019, on = 'idplug_station')

another_dataset = another_dataset.rename(columns={'Gis_X':'Gis_Xidplug',
       'Gis_Y':'Gis_Yidplug', 'Fecha de Alta':'Fecha de Alta_idplug', 'Distrito':'Distrito_idplug', 'Barrio':'Barrio_idplug', 'Calle':'Calle_idplug', 'Nº Finca':'Nº Finca_idplug',
        'Anclajes':'Anclajes_idplug', 'Longitud':'Longitud_idplug', 'Latitud':'Latitud_idplug', 'DIRECCION':'DIRECCION_idplug',
       'Year':'Year_idplug', 'Locationlist':'Locationlist_idplug', 'X':'X_idplug', 'Y':'Y_idplug'})
del another_dataset['ids']

another_dataset.to_csv("another_dataset")
new_dataset.to_csv("new_dataset")# to save
new_dataset = pd.merge(jan18, stations_2019, on='idunplug_station') #join jan18 and stations_2019 using idunplug_station

join_data = pd.merge(new_dataset, another_dataset, on=['user_day_code'])

In [26]:
# another_dataset = pd.merge(jan18,stations_2019, on = 'idplug_station', indicator=True, how='left') #que tipo de merge hace en cada fila 
another_dataset.head(2)

,id_x,user_day_code,idunplug_station_x,idunplug_base,idplug_station,idplug_base,user_type,travel_time,unplug_hourTime,ageRange,zip_code,id_y,Gis_Xidplug,Gis_Yidplug,Fecha de Alta_idplug,Distrito_idplug,Barrio_idplug,Calle_idplug,Nº Finca_idplug,Tipo de Reserva,Anclajes_idplug,LONGITUD,LATITUD,DIRECCION_idplug,Year_idplug,idunplug_station_y
0,5a505acd2f384319304ed614,b00665a845be18ed9f036c2d72def1ef2fa2c49365cb64...,6,14,7,1,1,284,{'$date': '2018-01-01T00:00:00.000+0100'},5,28010,7,"440811,97","4475187,49",23/06/2014,01 CENTRO,01-04 JUSTICIA,"HORTALEZA, CALLE, DE",75,BiciMAD,21,-3.69774,40.425216,"CALLE DE HORTALEZA, 75",2014,7
1,5a505acf2f384319304ed839,9ff0154193786639eb616b117c1074bcdd2f3c816bd2e4...,166,8,7,10,1,1794,{'$date': '2018-01-01T01:00:00.000+0100'},0,,7,"440811,97","4475187,49",23/06/2014,01 CENTRO,01-04 JUSTICIA,"HORTALEZA, CALLE, DE",75,BiciMAD,21,-3.69774,40.425216,"CALLE DE HORTALEZA, 75",2014,7


### **Unified data of Jan18 and Stations 2019**

#### Load of Libraries

In [27]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import sklearn 

#### Acquiring data

In [28]:
data = pd.read_json("/content/drive/My Drive/Colab Notebooks/data/join_data", encoding="utf-8", dtype="frame")

In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2926629 entries, 0 to 2926628
Data columns (total 17 columns):
 #   Column             Dtype  
---  ------             -----  
 0   id_x_x             object 
 1   user_day_code      object 
 2   idunplug_station   int64  
 3   Distrito_idunplug  object 
 4   Barrio_idunplug    object 
 5   Longitud_idunplug  float64
 6   Latitud_idunplug   float64
 7   idplug_station_x   int64  
 8   Distrito_idplug    object 
 9   Barrio_idplug      object 
 10  Longitud_idplug    float64
 11  Latitud_idplug     float64
 12  user_type_x        int64  
 13  travel_time_x      int64  
 14  unplug_hourTime_x  object 
 15  ageRange_x         int64  
 16  zip_code_x         object 
dtypes: float64(4), int64(5), object(8)
memory usage: 401.9+ MB


In [30]:
# missing_values_count = data.isnull().sum().sum()
# missing_values_count

In [31]:
# Cleaning columns 
# data.columns
data = data.rename(columns={'id_x_x': '_id','idplug_station_x':'idplug_station',
       'user_type_x':'user_type', 'travel_time_x':'travel_time',
       'unplug_hourTime_x':'unplug_hourTime', 'ageRange_x':'ageRange', 'zip_code_x':'zip_code'})

In [32]:
data.head()

,_id,user_day_code,idunplug_station,Distrito_idunplug,Barrio_idunplug,Longitud_idunplug,Latitud_idunplug,idplug_station,Distrito_idplug,Barrio_idplug,Longitud_idplug,Latitud_idplug,user_type,travel_time,unplug_hourTime,ageRange,zip_code
0,5a505acd2f384319304ed614,b00665a845be18ed9f036c2d72def1ef2fa2c49365cb64...,6,01 CENTRO,01-04 JUSTICIA,-3.698409,40.424163,7,01 CENTRO,01-04 JUSTICIA,-3.697740,40.425216,1,284,{'$date': '2018-01-01T00:00:00.000+0100'},5,28010
1,5a505ad02f384319304ed925,38f477ef63c0d2f1cfcc673e4a5b9054512c51f1756b3f...,6,01 CENTRO,01-04 JUSTICIA,-3.698409,40.424163,65,01 CENTRO,01-03 CORTES,-3.693135,40.410829,1,824,{'$date': '2018-01-01T03:00:00.000+0100'},2,28007
2,5a505ad02f384319304ed925,38f477ef63c0d2f1cfcc673e4a5b9054512c51f1756b3f...,6,01 CENTRO,01-04 JUSTICIA,-3.698409,40.424163,65,03 RETIRO,03-05 JERÓNIMOS,-3.689382,40.416695,1,824,{'$date': '2018-01-01T03:00:00.000+0100'},2,28007
3,5a505adb2f384319304ee2dd,38f477ef63c0d2f1cfcc673e4a5b9054512c51f1756b3f...,104,04 SALAMANCA,04-01 RECOLETOS,-3.689579,40.426766,67,01 CENTRO,01-03 CORTES,-3.693135,40.410829,1,337,{'$date': '2018-01-01T17:00:00.000+0100'},2,28007
4,5a505adb2f384319304ee2dd,38f477ef63c0d2f1cfcc673e4a5b9054512c51f1756b3f...,104,04 SALAMANCA,04-01 RECOLETOS,-3.689579,40.426766,67,03 RETIRO,03-05 JERÓNIMOS,-3.689382,40.416695,1,337,{'$date': '2018-01-01T17:00:00.000+0100'},2,28007


In [33]:
# data['_id'].value_count()

In [34]:
data[data['_id'] == '5a505acd2f384319304ed614']

,_id,user_day_code,idunplug_station,Distrito_idunplug,Barrio_idunplug,Longitud_idunplug,Latitud_idunplug,idplug_station,Distrito_idplug,Barrio_idplug,Longitud_idplug,Latitud_idplug,user_type,travel_time,unplug_hourTime,ageRange,zip_code
0,5a505acd2f384319304ed614,b00665a845be18ed9f036c2d72def1ef2fa2c49365cb64...,6,01 CENTRO,01-04 JUSTICIA,-3.698409,40.424163,7,01 CENTRO,01-04 JUSTICIA,-3.69774,40.425216,1,284,{'$date': '2018-01-01T00:00:00.000+0100'},5,28010
